In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import anndata
import os

from scvi.dataset import PbmcDataset
from scvi.models import AutoZIVAE
from scvi.inference import UnsupervisedTrainer

In [2]:
test_mode = False
n_epochs_all = None
save_path = "data/"

n_genes_to_keep = 100 if test_mode else 1000
pbmc = PbmcDataset(save_path=save_path, save_path_10X=os.path.join(save_path, "10X"))
pbmc.subsample_genes(new_n_genes=n_genes_to_keep)

[2020-06-25 12:50:11,675] INFO - scvi.dataset.dataset | File /Users/galen/scVI/data/gene_info_pbmc.csv already downloaded
[2020-06-25 12:50:11,676] INFO - scvi.dataset.dataset | File /Users/galen/scVI/data/pbmc_metadata.pickle already downloaded
[2020-06-25 12:50:11,704] INFO - scvi.dataset.dataset | File /Users/galen/scVI/data/10X/pbmc8k/filtered_gene_bc_matrices.tar.gz already downloaded
[2020-06-25 12:50:11,704] INFO - scvi.dataset.dataset10X | Preprocessing dataset
[2020-06-25 12:50:28,790] INFO - scvi.dataset.dataset10X | Finished preprocessing dataset
[2020-06-25 12:50:28,910] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2020-06-25 12:50:28,911] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2020-06-25 12:50:29,119] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-25 12:50:29,201] INFO - scvi.dataset.dataset | Downsampled from 8381 to 8381 cells
[2020-06-25 12:50:29,252] INFO - scvi.dataset.dataset | File /Users/galen

/Users/galen/anaconda3/envs/scvi/lib/python3.6/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


[2020-06-25 12:50:39,812] INFO - scvi.dataset.dataset | Downsampling from 3346 to 1000 genes
[2020-06-25 12:50:39,871] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-25 12:50:39,886] INFO - scvi.dataset.dataset | Filtering non-expressing cells.
[2020-06-25 12:50:39,919] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2020-06-25 12:50:39,925] INFO - scvi.dataset.dataset | Downsampled from 11990 to 11990 cells


In [3]:
from scvi.dataset.utils import setup_anndata

adata = pbmc.to_anndata()
setup_anndata(adata,batch_key = 'batch_indices', labels_key = 'cell_types')

[2020-06-25 12:50:40,019] INFO - scvi.dataset.utils | Using data from adata.X
[2020-06-25 12:50:40,020] INFO - scvi.dataset.utils | Using batches from adata.obs["batch_indices"]
[2020-06-25 12:50:40,021] INFO - scvi.dataset.utils | Using labels from adata.obs["cell_types"]
[2020-06-25 12:50:40,024] INFO - scvi.dataset.utils | Computing library size prior per batch
[2020-06-25 12:50:40,036] INFO - scvi.dataset.utils | Successfully registered anndata object containing 11990 cells, 1000 genes, and 2 batches 
Registered keys:['X', 'batch_indices', 'local_l_mean', 'local_l_var', 'labels']


/Users/galen/anaconda3/envs/scvi/lib/python3.6/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [4]:
adata

AnnData object with n_obs × n_vars = 11990 × 1000
    obs: 'barcodes', 'batch_indices', 'cell_types', '_scvi_labels', '_scvi_local_l_mean', '_scvi_local_l_var'
    uns: 'cell_measurements_col_mappings', 'scvi_data_registry', 'scvi_summary_stats'

In [5]:
from scvi.dataset.biodataset import BioDataset

In [7]:
autozivae = AutoZIVAE(n_input=1000, alpha_prior=0.5, beta_prior=0.5, minimal_dropout=0.01)
autozitrainer = UnsupervisedTrainer(autozivae, adata)
n_epochs = 2
autozitrainer.train(n_epochs=n_epochs, lr=1e-2)

[2020-06-25 12:52:04,199] INFO - scvi.inference.inference | KL warmup phase exceeds overall training phaseIf your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
[2020-06-25 12:52:04,200] INFO - scvi.inference.inference | KL warmup for 400 epochs



[2020-06-25 12:52:15,013] INFO - scvi.inference.inference | Training is still in warming up phase. If your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.


In [9]:
from scipy.stats import beta
outputs = autozivae.get_alphas_betas()
alpha_posterior = outputs['alpha_posterior']
beta_posterior = outputs['beta_posterior']

# Threshold (or Kzinb/Knb+Kzinb in paper)
threshold = 0.5

# q(delta_g < 0.5) probabilities
zi_probs = beta.cdf(0.5,alpha_posterior,beta_posterior)

# ZI genes
is_zi_pred = (zi_probs > threshold)

print('Fraction of predicted ZI genes :', is_zi_pred.mean())

Fraction of predicted ZI genes : 0.458


In [10]:
mask_sufficient_expression = (np.array(pbmc.X.mean(axis=0)) > 1.).reshape(-1)
print('Fraction of genes with avg expression > 1 :', mask_sufficient_expression.mean())
print('Fraction of predicted ZI genes with avg expression > 1 :', is_zi_pred[mask_sufficient_expression].mean())

Fraction of genes with avg expression > 1 : 0.061
Fraction of predicted ZI genes with avg expression > 1 : 0.4098360655737705


In [13]:
# Model definition
autozivae_genelabel = AutoZIVAE(n_input=pbmc.nb_genes, alpha_prior=0.5, beta_prior=0.5, minimal_dropout=0.01,
                         dispersion='gene-label', zero_inflation='gene-label', n_labels=pbmc.n_labels)
autozitrainer_genelabel = UnsupervisedTrainer(autozivae_genelabel, adata)

# Training
n_epochs = 2 if n_epochs_all is None else n_epochs_all
autozitrainer_genelabel.train(n_epochs=n_epochs, lr=1e-2)

# Retrieve posterior distribution parameters
outputs_genelabel = autozivae_genelabel.get_alphas_betas()
alpha_posterior_genelabel = outputs_genelabel['alpha_posterior']
beta_posterior_genelabel = outputs_genelabel['beta_posterior']

[2020-06-25 12:56:38,437] INFO - scvi.inference.inference | KL warmup phase exceeds overall training phaseIf your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.
[2020-06-25 12:56:38,438] INFO - scvi.inference.inference | KL warmup for 400 epochs



[2020-06-25 12:56:50,220] INFO - scvi.inference.inference | Training is still in warming up phase. If your applications rely on the posterior quality, consider training for more epochs or reducing the kl warmup.


In [14]:
# q(delta_g < 0.5) probabilities
zi_probs_genelabel = beta.cdf(0.5,alpha_posterior_genelabel,beta_posterior_genelabel)

# ZI gene-cell-types
is_zi_pred_genelabel = (zi_probs_genelabel > threshold)

for ind_cell_type, cell_type in enumerate(pbmc.cell_types):

    is_zi_pred_genelabel_here = is_zi_pred_genelabel[:,ind_cell_type]
    print('Fraction of predicted ZI genes for cell type {} :'.format(cell_type),
          is_zi_pred_genelabel_here.mean(),'\n')

Fraction of predicted ZI genes for cell type B cells : 0.464 

Fraction of predicted ZI genes for cell type CD14+ Monocytes : 0.486 

Fraction of predicted ZI genes for cell type CD4 T cells : 0.502 

Fraction of predicted ZI genes for cell type CD8 T cells : 0.523 

Fraction of predicted ZI genes for cell type Dendritic Cells : 0.5 

Fraction of predicted ZI genes for cell type FCGR3A+ Monocytes : 0.48 

Fraction of predicted ZI genes for cell type Megakaryocytes : 0.562 

Fraction of predicted ZI genes for cell type NK cells : 0.564 

Fraction of predicted ZI genes for cell type Other : 0.478 



In [15]:
# With avg expressions > 1
for ind_cell_type, cell_type in enumerate(pbmc.cell_types):
    mask_sufficient_expression = (np.array(pbmc.X[pbmc.labels.reshape(-1) == ind_cell_type,:].mean(axis=0)) > 1.).reshape(-1)
    print('Fraction of genes with avg expression > 1 for cell type {} :'.format(cell_type),
          mask_sufficient_expression.mean())
    is_zi_pred_genelabel_here = is_zi_pred_genelabel[mask_sufficient_expression,ind_cell_type]
    print('Fraction of predicted ZI genes with avg expression > 1 for cell type {} :'.format(cell_type),
          is_zi_pred_genelabel_here.mean(), '\n')

Fraction of genes with avg expression > 1 for cell type B cells : 0.034
Fraction of predicted ZI genes with avg expression > 1 for cell type B cells : 0.35294117647058826 

Fraction of genes with avg expression > 1 for cell type CD14+ Monocytes : 0.08
Fraction of predicted ZI genes with avg expression > 1 for cell type CD14+ Monocytes : 0.3 

Fraction of genes with avg expression > 1 for cell type CD4 T cells : 0.037
Fraction of predicted ZI genes with avg expression > 1 for cell type CD4 T cells : 0.35135135135135137 

Fraction of genes with avg expression > 1 for cell type CD8 T cells : 0.051
Fraction of predicted ZI genes with avg expression > 1 for cell type CD8 T cells : 0.45098039215686275 

Fraction of genes with avg expression > 1 for cell type Dendritic Cells : 0.181
Fraction of predicted ZI genes with avg expression > 1 for cell type Dendritic Cells : 0.3425414364640884 

Fraction of genes with avg expression > 1 for cell type FCGR3A+ Monocytes : 0.126
Fraction of predicted Z